In [ ]:
!pip install seaborn scipy

In [ ]:
from copick.impl.filesystem import CopickRootFSSpec
COPICK_CONFIG_PATH = "../assets/samba_config_jfinder.json"
root = CopickRootFSSpec.from_file(COPICK_CONFIG_PATH)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import copy
import random

from scipy.stats import ks_2samp



class SplitDataset:
    def __init__(self, config_file: str):
        self.root = CopickRootFSSpec.from_file(config_file)
        self.arrs = []
        self.tomograms = []

        N = len(self.root.runs)
        self.particle_map = {o.name: i for i,o in enumerate(self.root.config.pickable_objects)}
        self.run_stats_list = [0]*N
        for i, run in enumerate(self.root.runs[:N]):
            self.tomograms.append(run.name)
            counter = defaultdict(int)
            percent = defaultdict(float)
            all = 0
            for pick in run.picks:
                if pick.points is not None:
                    counter[pick.pickable_object_name] = len(pick.points)
                    all += len(pick.points)

            
            for k in self.particle_map.keys():
                if k not in counter:
                    counter[k] = 0
            
            # for k,v in counter.items():
            #     percent[k] = v/all
            percent = counter
            
            percent = {k: percent[k] for k in sorted(percent.keys())}
            print(i, run.name, percent)
            self.run_stats_list[i] = percent

        print(f'self.run_stats_list {len(self.run_stats_list)}')
        for d in self.run_stats_list:
            arr = []
            for k,v in d.items():
                arr += [self.particle_map[k]]*v
            self.arrs.append(arr)
        
        
    def plot_all_distributions(self):
        # Choose a colormap
        colormap = plt.cm.viridis
        # Generate a list of colors from the colormap
        colors = [colormap(i / len(self.arrs)) for i in range(len(self.arrs))]

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 1, 1)
        for i,arr in enumerate(self.arrs):  
            sns.histplot(arr, kde=True, color=colors[i]) #, label=self.tomograms[i])
        plt.legend()
        plt.title('Histogram')

        # plt.subplot(1, 2, 2)
        # sns.boxplot(data=self.arrs)
        # plt.xticks([i for i in range(len(self.arrs))], [f'{i}' for i in range(len(self.arrs))])
        # plt.title('Box Plot')

        plt.show()

    
    def plot_2dist(self, first: int, second: int, arr=False):
        if arr:
            arr1 = first
            arr2 = second
            label1 = 'arr1'
            label2 = 'arr2'
        else:
            arr1 = self.arrs[first]
            arr2 = self.arrs[second]
            label1 = self.tomograms[first]
            label2 = self.tomograms[second]

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 1, 1)
        sns.histplot(arr1, kde=True, color='blue', label=label1)
        sns.histplot(arr2, kde=True, color='red', label=label2)
        plt.legend()
        plt.title('Histogram')

        # plt.subplot(1, 2, 2)
        # sns.boxplot(data=[arr1, arr2])
        # plt.xticks([0, 1], [label1, label2])
        # plt.title('Box Plot')

        plt.show()
    
    
    def plot_dist_3arrs(self, arr1, arr2, arr3):
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 1, 1)
        sns.histplot(arr1, kde=True, color='blue', label='training_dataset')
        sns.histplot(arr2, kde=True, color='red', label='test_dataset1')
        sns.histplot(arr3, kde=True, color='green', label='test_dataset2')
        plt.legend()
        plt.title('Histogram')
        plt.show()
    
    def plot_dist_test(self, arr2, arr3):
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 1, 1)
        sns.histplot(arr2, kde=True, color='red', label='test_dataset1')
        sns.histplot(arr3, kde=True, color='green', label='test_dataset2')
        plt.legend()
        plt.title('Histogram')
        plt.show()
    
    def is_close_dist(self, first: int, second: int, threshold=0.05):
        return self.is_arr_close_dist(self.arrs[first], self.arrs[second], threshold)
    
    
    @staticmethod
    def is_arr_close_dist(arr1, arr2, threshold=0.05):
        # Kolmogorov-Smirnov test
        ks_stat, ks_p_value = ks_2samp(arr1, arr2)

        # If the p-value is high (e.g., > 0.05), it suggests that there is no significant difference between the distributions of the two dictionaries' values.
        print(f"KS Statistic: {ks_stat}, P-value: {ks_p_value}")
        return ks_p_value > threshold
    
    
    def make_buckets(self, threshold=0.05):
        if len(self.run_stats_list):
            stats = copy.deepcopy(self.run_stats_list[0])
            self.buckets = [[stats, set([0])]] # list of [defaultdict, set()]
        
        add_new = True
        for i in range(1, len(self.arrs)):
            for j in range(len(self.buckets)):
                arr1 = self.arrs[i]
                arr2 = []
                for k,v in self.buckets[j][0].items():
                    arr2 += [self.particle_map[k]]*v  
                if self.is_arr_close_dist(arr1, arr2, threshold):
                    self.buckets[j][0] = {k: self.buckets[j][0][k] + self.run_stats_list[i][k] for k in self.run_stats_list[i].keys()}
                    self.buckets[j][1].add(i)
                    add_new = False
                    break
                else:
                    add_new = True
            
            if add_new:  
                stats = copy.deepcopy(self.run_stats_list[i])
                self.buckets.append([stats, set([i])])

        
        for bucket in self.buckets:
            print(bucket)
        
        print(f'{len(self.buckets)} buckets')


        # Choose a colormap
        colormap = plt.cm.viridis
        # Generate a list of colors from the colormap
        colors = [colormap(i / len(self.buckets)) for i in range(len(self.buckets))]
        # visualizing buckets
        plt.subplot(1, 1, 1)
        for i,bucket in enumerate(self.buckets):
            arr = []
            for k,v in bucket[0].items():
                arr += [self.particle_map[k]]*v
            self.arrs.append(arr)  
            sns.histplot(arr, kde=True, color=colors[i], label=f'cluster {i}')
        plt.legend()
        plt.title('Histogram') 
        

    
    def random_split_list(self, my_list, ks=[0.6, 0.2, 0.2]):
        # Shuffle the original list to ensure randomness
        random.shuffle(my_list)
        
        # Calculate split indices
        train = round(ks[0] * len(my_list))
        test1 = round(ks[1] * len(my_list))
        test2 = round(ks[2] * len(my_list))
        
        # Split the list into three parts
        train_set = my_list[:train]
        test_set1 = my_list[train:train+test1]
        test_set2 = my_list[train+test1:train+test1+test2]
        test_set3 = my_list[train+test1+test2:]
        
        return train_set, test_set1, test_set2, test_set3

    
    def id2arr(self, ids):
        arr = []
        for i in ids:
            arr = arr + self.arrs[i]
        return arr

    def generate_datasets(self, ks=[0.6, 0.2, 0.2]):
        #self.make_buckets()
        train_dt = [] 
        test_dt1 = []
        test_dt2 = []
        test_dt3 = []
        for bucket in self.buckets:
            train_set, test_set1, test_set2, test_set3 = self.random_split_list(list(bucket[1]), ks)
            train_dt = train_dt + train_set
            test_dt1 = test_dt1 + test_set1
            test_dt2 = test_dt2 + test_set2
            test_dt3 = test_dt3 + test_set3

        train_dataset = [self.tomograms[i] for i in train_dt] 
        test_dataset1 = [self.tomograms[i] for i in test_dt1]
        test_dataset2 = [self.tomograms[i] for i in test_dt2] 
        test_dataset3 = [self.tomograms[i] for i in test_dt3] 
 
        train_arr = self.id2arr(train_dt)
        test_arr1 = self.id2arr(test_dt1)
        test_arr2 = self.id2arr(test_dt2)
        self.plot_dist_3arrs(train_arr, test_arr1, test_arr2) 
        self.plot_dist_test(test_arr1, test_arr2) 
        return train_dataset, test_dataset1, test_dataset2, test_dataset3


In [ ]:
datasets = SplitDataset(COPICK_CONFIG_PATH)

In [ ]:
datasets.plot_all_distributions()

In [ ]:
datasets.make_buckets()

In [ ]:
train_dataset, test_dataset1, test_dataset2, test_dataset3 = datasets.generate_datasets([0.07, 0.03, 0.45, 0.45])
print(f'train_dataset\n{train_dataset}\ntest_dataset1\n{test_dataset1}\ntest_dataset2\n{test_dataset2}test_dataset2\n{test_dataset3}')
print(len(train_dataset), len(test_dataset1), len(test_dataset2), len(test_dataset3))

## Evaluate the datasets

```
for each fake model in [sigma = 0, sigma = 1, sigma = 10]:
    for each tomogram:
        for each particle type:
            newpicks = []
            for each point in ground truth/jfinder picks:
                point[0] += gaussian(sigma)
                point[1] += gaussian(sigma)
                point[2] += gaussian(sigma)

                point += gaussian3D(sigma)
                # TODO add clipping to check that points stay within tomogram dimensions
                newpicks.append(point)
```

In [ ]:
# import math

# def gaussian_function(x, mu=0.0, sigma=1.0):
#     """
#     Calculate the value of the Gaussian function for a given x, mean (mu), and standard deviation (sigma).
    
#     Parameters:
#     x (float): The value at which to evaluate the Gaussian function.
#     mu (float): The mean of the Gaussian distribution.
#     sigma (float): The standard deviation of the Gaussian distribution.
    
#     Returns:
#     float: The value of the Gaussian function at x.
#     """
#     coefficient = 1.0 / (sigma * math.sqrt(2 * math.sqrt(math.pi)))
#     exponent = -((x - mu) ** 2) / (2 * sigma ** 2)
#     return coefficient * math.exp(exponent)

# # Example usage:
# x_value = 1.0

# result = gaussian_function(x_value)
# print("The value of the Gaussian function at x =", x_value, "is", result)

In [ ]:
def Guassian3d(uncertainty=30):
    mean = [0, 0, 0]
    covariance_matrix = [[1, 0.5, 0.5], 
                        [0.5, 1, 0.5], 
                        [0.5, 0.5, 1]]
    return np.random.multivariate_normal(mean, covariance_matrix) * uncertainty

Guassian3d()

In [ ]:
test1_1 = ['TS_37_6', 'TS_4_3', 'TS_85_2', 'TS_2_2', 'TS_102_4', 'TS_100_3', 'TS_94_8', 'TS_12_3', 'TS_88_9', 'TS_96_7', 'TS_11_1', 'TS_86_9', 'TS_93_5', 'TS_87_9', 'TS_69_8', 'TS_89_3', 'TS_86_2', 'TS_69_4', 'TS_65_9', 'TS_80_5', 'TS_3_3', 'TS_77_5', 'TS_30_9', 'TS_104_2', 'TS_67_4', 'TS_76_8', 'TS_39_5', 'TS_78_5', 'TS_81_8', 'TS_99_4', 'TS_15_3', 'TS_81_1', 'TS_104_9', 'TS_111_5', 'TS_100_7', 'TS_65_2', 'TS_87_5', 'TS_47_4', 'TS_65_8', 'TS_5_4', 'TS_71_3', 'TS_48_4', 'TS_35_9', 'TS_34_2', 'TS_34_5', 'TS_46_1', 'TS_7_2', 'TS_30_2', 'TS_47_6', 'TS_15_2', 'TS_26_4', 'TS_9_3', 'TS_34_7', 'TS_94_6', 'TS_63_7', 'TS_109_6', 'TS_34_8', 'TS_49_5', 'TS_27_7', 'TS_45_4', 'TS_44_2', 'TS_46_9', 'TS_26_7', 'TS_110_8', 'TS_25_5', 'TS_107_5', 'TS_108_1', 'TS_102_7', 'TS_106_4', 'TS_48_5', 'TS_103_3', 'TS_72_8', 'TS_90_5', 'TS_69_2', 'TS_29_7', 'TS_58_1', 'TS_87_6', 'TS_96_8', 'TS_14_8', 'TS_68_7', 'TS_9_8', 'TS_13_9', 'TS_95_6', 'TS_78_9', 'TS_11_3', 'TS_78_1', 'TS_3_4', 'TS_83_1', 'TS_59_7', 'TS_43_7', 'TS_86_3', 'TS_35_7', 'TS_99_1', 'TS_43_3', 'TS_45_2', 'TS_35_3', 'TS_71_8', 'TS_39_1', 'TS_4_4']
test1_2 = ['TS_94_7', 'TS_82_3', 'TS_26_1', 'TS_81_6', 'TS_69_7', 'TS_52_5', 'TS_111_2', 'TS_112_5', 'TS_63_5', 'TS_93_3', 'TS_15_1', 'TS_24_3', 'TS_90_3', 'TS_70_4', 'TS_61_8', 'TS_100_9', 'TS_82_4', 'TS_58_8', 'TS_47_2', 'TS_110_3', 'TS_109_8', 'TS_96_2', 'TS_14_5', 'TS_93_2', 'TS_89_1', 'TS_120_2', 'TS_58_9', 'TS_97_6', 'TS_92_4', 'TS_100_4', 'TS_115_5', 'TS_64_9', 'TS_114_2', 'TS_12_2', 'TS_99_2', 'TS_95_8', 'TS_4_5', 'TS_24_6', 'TS_80_8', 'TS_6_8', 'TS_5_8', 'TS_116_5', 'TS_68_5', 'TS_96_5', 'TS_31_1', 'TS_47_3', 'TS_115_8', 'TS_9_9', 'TS_86_8', 'TS_14_6', 'TS_118_7', 'TS_93_4', 'TS_25_6', 'TS_25_2', 'TS_109_2', 'TS_106_5', 'TS_114_1', 'TS_105_2', 'TS_87_8', 'TS_103_1', 'TS_89_8', 'TS_12_9', 'TS_28_3', 'TS_45_5', 'TS_8_7', 'TS_104_1', 'TS_24_4', 'TS_104_6', 'TS_97_3', 'TS_109_3', 'TS_83_5', 'TS_74_8', 'TS_91_5', 'TS_74_1', 'TS_72_2', 'TS_77_1', 'TS_74_5', 'TS_85_8', 'TS_96_4', 'TS_69_1', 'TS_107_3', 'TS_95_1', 'TS_90_6', 'TS_93_9', 'TS_7_4', 'TS_95_5', 'TS_68_9', 'TS_114_3', 'TS_39_7', 'TS_52_3', 'TS_86_1', 'TS_60_5', 'TS_44_8', 'TS_37_4', 'TS_87_1', 'TS_59_3', 'TS_62_7', 'TS_71_4', 'TS_84_8']

test2_1 = ['TS_115_7', 'TS_92_4', 'TS_67_3', 'TS_87_9', 'TS_4_6', 'TS_64_9', 'TS_86_5', 'TS_93_5', 'TS_96_3', 'TS_78_2', 'TS_9_2', 'TS_68_4', 'TS_98_1', 'TS_12_2', 'TS_112_6', 'TS_104_9', 'TS_98_5', 'TS_110_3', 'TS_86_9', 'TS_56_7', 'TS_76_8', 'TS_120_2', 'TS_26_1', 'TS_99_2', 'TS_93_3', 'TS_30_9', 'TS_1_7', 'TS_102_2', 'TS_76_4', 'TS_89_1', 'TS_99_6', 'TS_67_2', 'TS_115_1', 'TS_62_5', 'TS_3_5', 'TS_15_3', 'TS_88_1', 'TS_58_9', 'TS_99_3', 'TS_102_1', 'TS_94_5', 'TS_111_5', 'TS_2_6', 'TS_95_9', 'TS_80_5', 'TS_81_8', 'TS_97_6', 'TS_24_3', 'TS_63_9', 'TS_83_6', 'TS_31_8', 'TS_98_7', 'TS_85_1', 'TS_7_8', 'TS_12_3', 'TS_96_5', 'TS_116_5', 'TS_52_4', 'TS_27_1', 'TS_115_4', 'TS_47_3', 'TS_60_8', 'TS_46_1', 'TS_89_6', 'TS_6_4', 'TS_82_1', 'TS_25_2', 'TS_63_6', 'TS_83_3', 'TS_33_4', 'TS_65_8', 'TS_100_6', 'TS_49_6', 'TS_98_9', 'TS_49_2', 'TS_2_5', 'TS_76_9', 'TS_15_2', 'TS_9_3', 'TS_7_2', 'TS_105_5', 'TS_26_7', 'TS_73_6', 'TS_1_1', 'TS_25_7', 'TS_101_9', 'TS_34_8', 'TS_27_9', 'TS_103_4', 'TS_28_5', 'TS_1_2', 'TS_94_3', 'TS_47_7', 'TS_109_6', 'TS_48_9', 'TS_37_1', 'TS_49_5', 'TS_107_5', 'TS_110_2', 'TS_109_9', 'TS_109_3', 'TS_108_1', 'TS_102_7', 'TS_103_2', 'TS_27_5', 'TS_104_3', 'TS_84_7', 'TS_74_1', 'TS_117_7', 'TS_72_2', 'TS_69_2', 'TS_91_5', 'TS_99_9', 'TS_79_4', 'TS_96_8', 'TS_78_1', 'TS_61_7', 'TS_91_9', 'TS_90_4', 'TS_69_1', 'TS_82_2', 'TS_80_1', 'TS_67_6', 'TS_14_1', 'TS_83_1', 'TS_64_3', 'TS_85_8', 'TS_74_3', 'TS_60_4', 'TS_78_9', 'TS_108_4', 'TS_70_2', 'TS_25_4', 'TS_46_8', 'TS_5_9', 'TS_71_9', 'TS_60_3', 'TS_86_3', 'TS_35_7', 'TS_43_9', 'TS_25_1', 'TS_44_5', 'TS_6_6', 'TS_59_3', 'TS_39_1', 'TS_73_7', 'TS_4_4']
test2_2 = ['TS_11_1', 'TS_91_7', 'TS_37_2', 'TS_104_2', 'TS_47_8', 'TS_13_5', 'TS_93_2', 'TS_89_3', 'TS_65_6', 'TS_98_3', 'TS_87_2', 'TS_109_4', 'TS_60_1', 'TS_71_2', 'TS_70_4', 'TS_58_8', 'TS_61_5', 'TS_31_3', 'TS_78_5', 'TS_5_5', 'TS_97_4', 'TS_65_9', 'TS_112_5', 'TS_87_5', 'TS_2_4', 'TS_109_8', 'TS_67_7', 'TS_94_4', 'TS_100_7', 'TS_96_7', 'TS_97_7', 'TS_79_9', 'TS_75_2', 'TS_70_3', 'TS_12_5', 'TS_63_2', 'TS_4_3', 'TS_94_7', 'TS_61_6', 'TS_60_6', 'TS_91_2', 'TS_115_5', 'TS_85_6', 'TS_25_3', 'TS_81_4', 'TS_114_2', 'TS_1_4', 'TS_61_1', 'TS_88_3', 'TS_108_8', 'TS_63_5', 'TS_99_7', 'TS_11_9', 'TS_102_4', 'TS_100_4', 'TS_34_5', 'TS_108_3', 'TS_109_5', 'TS_80_8', 'TS_34_7', 'TS_5_3', 'TS_108_6', 'TS_115_8', 'TS_30_2', 'TS_86_8', 'TS_32_5', 'TS_30_7', 'TS_45_6', 'TS_95_3', 'TS_2_3', 'TS_6_8', 'TS_111_8', 'TS_71_3', 'TS_5_8', 'TS_70_5', 'TS_93_8', 'TS_35_9', 'TS_93_4', 'TS_111_9', 'TS_115_6', 'TS_26_6', 'TS_46_9', 'TS_44_2', 'TS_4_9', 'TS_106_5', 'TS_48_2', 'TS_25_5', 'TS_12_9', 'TS_44_9', 'TS_6_1', 'TS_45_5', 'TS_101_5', 'TS_101_1', 'TS_98_6', 'TS_103_5', 'TS_27_4', 'TS_87_8', 'TS_61_4', 'TS_108_9', 'TS_104_1', 'TS_24_4', 'TS_75_7', 'TS_106_4', 'TS_48_5', 'TS_104_6', 'TS_25_8', 'TS_83_5', 'TS_59_2', 'TS_80_6', 'TS_88_7', 'TS_58_7', 'TS_74_8', 'TS_90_2', 'TS_62_1', 'TS_87_4', 'TS_68_7', 'TS_93_9', 'TS_11_3', 'TS_11_7', 'TS_70_7', 'TS_107_4', 'TS_96_4', 'TS_13_9', 'TS_95_5', 'TS_112_4', 'TS_63_3', 'TS_95_6', 'TS_1_8', 'TS_14_4', 'TS_75_1', 'TS_59_5', 'TS_77_1', 'TS_7_4', 'TS_44_3', 'TS_87_3', 'TS_39_7', 'TS_116_9', 'TS_72_3', 'TS_86_1', 'TS_44_4', 'TS_43_3', 'TS_29_9', 'TS_59_8', 'TS_37_4', 'TS_58_6', 'TS_87_1', 'TS_59_1', 'TS_70_8', 'TS_62_7', 'TS_71_4', 'TS_84_8']

test3_1 = ['TS_78_5', 'TS_25_3', 'TS_115_7', 'TS_98_7', 'TS_7_8', 'TS_81_4', 'TS_63_2', 'TS_70_4', 'TS_81_1', 'TS_9_2', 'TS_109_8', 'TS_99_6', 'TS_90_3', 'TS_37_6', 'TS_12_3', 'TS_81_6', 'TS_114_2', 'TS_111_5', 'TS_82_5', 'TS_64_9', 'TS_1_3', 'TS_14_5', 'TS_24_3', 'TS_67_7', 'TS_60_7', 'TS_96_2', 'TS_77_6', 'TS_59_6', 'TS_3_3', 'TS_80_5', 'TS_89_3', 'TS_69_8', 'TS_97_2', 'TS_67_4', 'TS_111_1', 'TS_108_8', 'TS_30_9', 'TS_65_9', 'TS_98_2', 'TS_62_8', 'TS_110_3', 'TS_78_2', 'TS_81_8', 'TS_96_3', 'TS_76_3', 'TS_12_5', 'TS_98_1', 'TS_108_7', 'TS_88_3', 'TS_94_4', 'TS_52_5', 'TS_13_5', 'TS_111_2', 'TS_7_1', 'TS_88_1', 'TS_15_9', 'TS_4_3', 'TS_89_2', 'TS_93_5', 'TS_77_2', 'TS_97_6', 'TS_92_4', 'TS_102_1', 'TS_85_2', 'TS_68_4', 'TS_86_9', 'TS_85_1', 'TS_89_1', 'TS_69_4', 'TS_86_4', 'TS_71_5', 'TS_93_3', 'TS_5_5', 'TS_67_3', 'TS_115_8', 'TS_92_8', 'TS_111_9', 'TS_105_5', 'TS_31_6', 'TS_14_6', 'TS_76_9', 'TS_35_9', 'TS_26_4', 'TS_93_8', 'TS_111_8', 'TS_2_3', 'TS_9_9', 'TS_5_4', 'TS_30_2', 'TS_6_4', 'TS_93_6', 'TS_80_8', 'TS_25_6', 'TS_24_8', 'TS_95_3', 'TS_115_6', 'TS_45_1', 'TS_33_4', 'TS_34_5', 'TS_42_7', 'TS_1_9', 'TS_116_5', 'TS_4_7', 'TS_118_7', 'TS_46_5', 'TS_68_5', 'TS_100_6', 'TS_60_8', 'TS_27_7', 'TS_12_9', 'TS_25_5', 'TS_89_8', 'TS_44_7', 'TS_110_8', 'TS_1_2', 'TS_87_8', 'TS_44_9', 'TS_4_9', 'TS_53_7', 'TS_101_9', 'TS_73_6', 'TS_9_4', 'TS_26_7', 'TS_106_1', 'TS_25_7', 'TS_27_4', 'TS_34_8', 'TS_103_4', 'TS_47_7', 'TS_48_2', 'TS_24_5', 'TS_109_3', 'TS_102_7', 'TS_9_5', 'TS_12_1', 'TS_75_7', 'TS_25_8', 'TS_84_5', 'TS_104_3', 'TS_108_1', 'TS_24_4', 'TS_106_4', 'TS_99_9', 'TS_5_1', 'TS_29_7', 'TS_72_8', 'TS_74_1', 'TS_90_2', 'TS_80_6', 'TS_90_5', 'TS_58_7', 'TS_87_4', 'TS_74_8', 'TS_75_1', 'TS_90_4', 'TS_96_4', 'TS_78_1', 'TS_85_8', 'TS_71_7', 'TS_91_9', 'TS_90_6', 'TS_60_4', 'TS_63_3', 'TS_1_8', 'TS_70_1', 'TS_78_8', 'TS_67_6', 'TS_9_8', 'TS_61_7', 'TS_82_2', 'TS_11_3', 'TS_59_7', 'TS_69_5', 'TS_70_7', 'TS_78_9', 'TS_107_3', 'TS_91_3', 'TS_114_3', 'TS_44_3', 'TS_99_1', 'TS_71_9', 'TS_86_3', 'TS_35_7', 'TS_60_5', 'TS_116_9', 'TS_39_7', 'TS_43_2', 'TS_43_9', 'TS_43_3', 'TS_43_5', 'TS_35_3', 'TS_29_9', 'TS_39_1', 'TS_59_1', 'TS_81_2', 'TS_62_4', 'TS_87_7']
test3_2 = ['TS_31_8', 'TS_63_5', 'TS_15_7', 'TS_82_4', 'TS_109_4', 'TS_89_5', 'TS_88_8', 'TS_100_4', 'TS_76_4', 'TS_15_8', 'TS_65_4', 'TS_113_4', 'TS_94_7', 'TS_3_5', 'TS_70_9', 'TS_15_1', 'TS_6_9', 'TS_112_6', 'TS_98_4', 'TS_94_5', 'TS_70_3', 'TS_85_6', 'TS_112_5', 'TS_90_7', 'TS_2_2', 'TS_93_2', 'TS_98_3', 'TS_61_1', 'TS_99_4', 'TS_100_3', 'TS_62_5', 'TS_12_2', 'TS_37_2', 'TS_47_8', 'TS_56_7', 'TS_94_1', 'TS_77_5', 'TS_83_6', 'TS_104_2', 'TS_79_9', 'TS_59_4', 'TS_86_5', 'TS_61_5', 'TS_1_5', 'TS_98_5', 'TS_89_4', 'TS_61_2', 'TS_120_2', 'TS_104_9', 'TS_76_8', 'TS_60_2', 'TS_11_1', 'TS_2_6', 'TS_68_8', 'TS_106_7', 'TS_102_2', 'TS_71_2', 'TS_1_4', 'TS_65_6', 'TS_91_7', 'TS_76_5', 'TS_100_7', 'TS_2_4', 'TS_1_7', 'TS_60_1', 'TS_102_8', 'TS_95_8', 'TS_4_6', 'TS_82_3', 'TS_107_1', 'TS_115_5', 'TS_100_9', 'TS_39_5', 'TS_46_1', 'TS_5_3', 'TS_86_8', 'TS_34_2', 'TS_109_5', 'TS_30_7', 'TS_5_8', 'TS_24_6', 'TS_71_3', 'TS_63_6', 'TS_45_6', 'TS_49_6', 'TS_108_3', 'TS_52_4', 'TS_70_5', 'TS_7_2', 'TS_49_2', 'TS_31_1', 'TS_94_6', 'TS_98_9', 'TS_92_2', 'TS_83_3', 'TS_47_4', 'TS_48_4', 'TS_96_9', 'TS_93_1', 'TS_27_1', 'TS_15_6', 'TS_6_8', 'TS_65_8', 'TS_89_6', 'TS_114_4', 'TS_92_1', 'TS_62_2', 'TS_26_6', 'TS_93_7', 'TS_101_1', 'TS_101_5', 'TS_44_2', 'TS_37_1', 'TS_106_5', 'TS_105_2', 'TS_63_7', 'TS_103_1', 'TS_103_5', 'TS_101_4', 'TS_103_9', 'TS_115_3', 'TS_46_9', 'TS_101_6', 'TS_27_9', 'TS_49_5', 'TS_28_5', 'TS_1_1', 'TS_109_2', 'TS_110_2', 'TS_8_7', 'TS_14_2', 'TS_101_7', 'TS_104_1', 'TS_27_5', 'TS_97_3', 'TS_48_5', 'TS_109_9', 'TS_108_9', 'TS_103_2', 'TS_79_4', 'TS_104_4', 'TS_79_5', 'TS_91_5', 'TS_116_6', 'TS_83_5', 'TS_72_2', 'TS_59_2', 'TS_78_4', 'TS_87_6', 'TS_117_7', 'TS_64_3', 'TS_95_1', 'TS_3_4', 'TS_114_5', 'TS_107_4', 'TS_77_1', 'TS_91_8', 'TS_74_3', 'TS_79_6', 'TS_112_4', 'TS_69_1', 'TS_13_9', 'TS_95_5', 'TS_95_6', 'TS_70_2', 'TS_7_4', 'TS_77_3', 'TS_74_5', 'TS_96_8', 'TS_83_1', 'TS_83_8', 'TS_108_4', 'TS_14_1', 'TS_68_9', 'TS_37_8', 'TS_46_8', 'TS_25_4', 'TS_72_3', 'TS_87_3', 'TS_72_7', 'TS_90_1', 'TS_5_9', 'TS_86_1', 'TS_86_6', 'TS_45_2', 'TS_44_4', 'TS_25_1', 'TS_6_6', 'TS_37_4', 'TS_63_8', 'TS_58_6', 'TS_73_7', 'TS_59_3', 'TS_70_8', 'TS_4_4', 'TS_62_7', 'TS_71_4', 'TS_84_8']

test4_1 = ['TS_52_5', 'TS_106_7', 'TS_76_6', 'TS_77_5', 'TS_31_3', 'TS_67_2', 'TS_15_7', 'TS_100_7', 'TS_67_3', 'TS_87_9', 'TS_97_2', 'TS_59_4', 'TS_100_9', 'TS_110_3', 'TS_90_3', 'TS_67_7', 'TS_24_3', 'TS_48_1', 'TS_69_8', 'TS_86_9', 'TS_64_9', 'TS_70_9', 'TS_94_5', 'TS_111_5', 'TS_90_7', 'TS_79_9', 'TS_93_5', 'TS_77_6', 'TS_65_6', 'TS_94_8', 'TS_12_2', 'TS_86_5', 'TS_6_7', 'TS_11_9', 'TS_60_7', 'TS_67_4', 'TS_87_5', 'TS_96_2', 'TS_96_3', 'TS_68_8', 'TS_89_1', 'TS_2_6', 'TS_99_3', 'TS_89_2', 'TS_115_5', 'TS_47_2', 'TS_9_2', 'TS_98_4', 'TS_77_2', 'TS_100_3', 'TS_70_3', 'TS_39_5', 'TS_86_2', 'TS_65_2', 'TS_109_4', 'TS_92_4', 'TS_102_1', 'TS_80_5', 'TS_94_1', 'TS_83_6', 'TS_60_2', 'TS_5_5', 'TS_31_8', 'TS_111_2', 'TS_120_2', 'TS_58_9', 'TS_7_1', 'TS_116_4', 'TS_71_5', 'TS_12_3', 'TS_97_6', 'TS_88_9', 'TS_15_1', 'TS_4_6', 'TS_25_3', 'TS_65_4', 'TS_14_5', 'TS_15_9', 'TS_4_5', 'TS_15_8', 'TS_61_1', 'TS_93_3', 'TS_82_4', 'TS_4_7', 'TS_26_4', 'TS_71_3', 'TS_105_5', 'TS_25_6', 'TS_93_6', 'TS_5_8', 'TS_15_6', 'TS_108_6', 'TS_96_5', 'TS_47_3', 'TS_96_9', 'TS_111_9', 'TS_65_8', 'TS_34_5', 'TS_93_8', 'TS_109_1', 'TS_47_6', 'TS_93_1', 'TS_45_1', 'TS_24_6', 'TS_27_1', 'TS_2_5', 'TS_46_1', 'TS_109_5', 'TS_83_3', 'TS_30_2', 'TS_5_4', 'TS_25_2', 'TS_111_8', 'TS_60_8', 'TS_15_2', 'TS_46_5', 'TS_49_6', 'TS_52_4', 'TS_35_9', 'TS_92_8', 'TS_89_6', 'TS_103_4', 'TS_89_8', 'TS_45_4', 'TS_26_6', 'TS_27_4', 'TS_73_6', 'TS_109_6', 'TS_103_1', 'TS_9_4', 'TS_27_9', 'TS_98_6', 'TS_47_7', 'TS_28_3', 'TS_25_7', 'TS_53_7', 'TS_114_1', 'TS_93_7', 'TS_106_5', 'TS_103_9', 'TS_1_2', 'TS_48_9', 'TS_12_9', 'TS_101_6', 'TS_115_3', 'TS_97_3', 'TS_103_2', 'TS_106_4', 'TS_24_4', 'TS_101_7', 'TS_24_5', 'TS_9_5', 'TS_108_1', 'TS_110_2', 'TS_109_3', 'TS_104_3', 'TS_25_8', 'TS_107_5', 'TS_62_1', 'TS_74_1', 'TS_74_8', 'TS_59_2', 'TS_72_8', 'TS_78_4', 'TS_87_4', 'TS_116_6', 'TS_5_1', 'TS_88_7', 'TS_104_4', 'TS_69_2', 'TS_99_9', 'TS_78_9', 'TS_61_7', 'TS_9_8', 'TS_67_8', 'TS_14_4', 'TS_95_1', 'TS_59_7', 'TS_70_1', 'TS_13_8', 'TS_91_3', 'TS_83_1', 'TS_82_2', 'TS_13_9', 'TS_93_9', 'TS_11_3', 'TS_95_6', 'TS_7_4', 'TS_59_5', 'TS_68_7', 'TS_14_8', 'TS_85_8', 'TS_91_8', 'TS_114_5', 'TS_77_3', 'TS_2_9', 'TS_69_1', 'TS_74_5', 'TS_43_7', 'TS_37_8', 'TS_46_8', 'TS_99_1', 'TS_72_3', 'TS_60_5', 'TS_86_6', 'TS_5_2', 'TS_39_7', 'TS_116_9', 'TS_72_7', 'TS_43_9', 'TS_44_8', 'TS_45_2', 'TS_43_2', 'TS_59_8', 'TS_6_6', 'TS_87_1', 'TS_58_6', 'TS_62_4', 'TS_71_8', 'TS_70_8']
test4_2 = ['TS_47_8', 'TS_76_4', 'TS_102_4', 'TS_76_8', 'TS_81_8', 'TS_37_6', 'TS_69_7', 'TS_37_2', 'TS_76_3', 'TS_98_1', 'TS_115_7', 'TS_112_5', 'TS_88_8', 'TS_30_9', 'TS_61_6', 'TS_69_4', 'TS_91_7', 'TS_89_3', 'TS_76_5', 'TS_75_2', 'TS_63_2', 'TS_97_7', 'TS_65_9', 'TS_99_7', 'TS_13_5', 'TS_108_7', 'TS_85_6', 'TS_91_2', 'TS_63_5', 'TS_94_4', 'TS_96_7', 'TS_99_6', 'TS_7_8', 'TS_97_4', 'TS_26_1', 'TS_107_1', 'TS_9_7', 'TS_78_5', 'TS_82_5', 'TS_63_9', 'TS_59_6', 'TS_93_2', 'TS_85_1', 'TS_88_3', 'TS_104_2', 'TS_12_5', 'TS_71_2', 'TS_58_8', 'TS_3_5', 'TS_102_8', 'TS_78_2', 'TS_98_5', 'TS_88_1', 'TS_114_2', 'TS_70_4', 'TS_86_4', 'TS_99_4', 'TS_95_8', 'TS_60_1', 'TS_81_1', 'TS_15_3', 'TS_89_5', 'TS_104_9', 'TS_62_5', 'TS_85_2', 'TS_89_4', 'TS_100_4', 'TS_81_4', 'TS_1_3', 'TS_3_3', 'TS_98_2', 'TS_94_7', 'TS_98_3', 'TS_99_2', 'TS_4_3', 'TS_109_8', 'TS_1_7', 'TS_2_2', 'TS_82_3', 'TS_60_6', 'TS_62_8', 'TS_11_1', 'TS_42_7', 'TS_70_5', 'TS_2_3', 'TS_47_4', 'TS_30_7', 'TS_6_4', 'TS_114_4', 'TS_32_3', 'TS_98_9', 'TS_95_3', 'TS_5_3', 'TS_1_9', 'TS_76_9', 'TS_33_4', 'TS_82_1', 'TS_48_4', 'TS_93_4', 'TS_116_5', 'TS_35_6', 'TS_31_1', 'TS_115_8', 'TS_68_5', 'TS_94_6', 'TS_34_2', 'TS_63_6', 'TS_86_8', 'TS_115_4', 'TS_27_3', 'TS_108_3', 'TS_31_6', 'TS_92_1', 'TS_34_7', 'TS_45_6', 'TS_92_2', 'TS_6_8', 'TS_14_6', 'TS_24_8', 'TS_32_5', 'TS_105_7', 'TS_44_2', 'TS_63_7', 'TS_105_2', 'TS_44_7', 'TS_106_1', 'TS_44_9', 'TS_28_5', 'TS_25_5', 'TS_4_9', 'TS_101_1', 'TS_110_8', 'TS_34_8', 'TS_101_9', 'TS_101_5', 'TS_46_9', 'TS_49_5', 'TS_6_1', 'TS_87_8', 'TS_103_5', 'TS_48_2', 'TS_1_1', 'TS_37_1', 'TS_94_3', 'TS_108_9', 'TS_103_3', 'TS_44_1', 'TS_75_7', 'TS_12_1', 'TS_84_5', 'TS_61_4', 'TS_104_1', 'TS_8_7', 'TS_1_6', 'TS_104_6', 'TS_14_2', 'TS_109_9', 'TS_90_2', 'TS_87_6', 'TS_73_8', 'TS_29_7', 'TS_79_4', 'TS_84_7', 'TS_91_5', 'TS_117_7', 'TS_83_5', 'TS_58_1', 'TS_72_2', 'TS_90_5', 'TS_90_4', 'TS_107_4', 'TS_96_8', 'TS_14_1', 'TS_70_7', 'TS_11_7', 'TS_77_1', 'TS_63_3', 'TS_78_8', 'TS_60_4', 'TS_90_6', 'TS_96_4', 'TS_95_5', 'TS_79_6', 'TS_80_1', 'TS_74_3', 'TS_3_4', 'TS_75_1', 'TS_108_4', 'TS_91_9', 'TS_68_9', 'TS_107_3', 'TS_83_8', 'TS_84_1', 'TS_64_3', 'TS_67_6', 'TS_112_4', 'TS_44_3', 'TS_25_4', 'TS_114_3', 'TS_86_3', 'TS_90_1', 'TS_35_7', 'TS_60_3', 'TS_87_3', 'TS_86_1', 'TS_71_9', 'TS_43_3', 'TS_43_5', 'TS_25_1', 'TS_44_4', 'TS_29_9', 'TS_35_3', 'TS_37_4', 'TS_73_7', 'TS_59_1', 'TS_59_3', 'TS_81_2', 'TS_39_1', 'TS_87_7', 'TS_4_4', 'TS_62_7', 'TS_71_4', 'TS_84_8']

test_datasets = [(test1_1, test1_2), (test2_1, test2_2), (test3_1, test3_2), (test4_1, test4_2)]

In [ ]:
Guassian3d()

## Save picks from Gaussian model

In [ ]:
# run = root.get_run('TS_1_1')
# picks = run.get_picks(object_name='thyroglobulin', user_id='deepfinder', session_id='0')
# print(picks)

for run in root.runs[:2]:
    # print(run)
    # _picks = run.get_picks(object_name='thyroglobulin', user_id='deepfinder', session_id='0')
    # print(_picks)
    for picks in run.picks:
        new_points = []
        if picks.points is not None and len(picks.points):
            for point in picks.points:
                wiggles = Guassian3d()
                point.location.x = min(point.location.x+wiggles[0], 6000)
                point.location.y = min(point.location.y+wiggles[1], 6000)
                point.location.z = min(point.location.z+wiggles[2], 1800)
                new_points.append(point)
            picks.points = new_points
            picks.store()


## Evaluation with the ground truth

In [ ]:
thresholds = {'apo-ferritin': 130, 
              'beta-galactosidase': 180, 
              'ribosome': 310,
              'thyroglobulin': 290,
              'virus-like-particle': 285,
              'junk': 200
            }  

In [ ]:
# from copick.impl.filesystem import CopickRootFSSpec
# root_gt = CopickRootFSSpec.from_file("../assets/samba_config_jfinder_ground_truth.json")
# root_gt.runs

In [ ]:
for tests in test_datasets:
    for test in tests:
        for name in test:
            run = root.get_run(name)
            for pick in run.picks:
                print(f'Jfinder {len(pick.points)}')
                break
                particle = pick.pickable_object_name
                if pick.points is not None and len(pick.points):
                    for point in pick.points:
                        wiggles = Guassian3d()
                        point.location.x += wiggles[0]
                        point.location.y += wiggles[1]
                        point.location.z += wiggles[2]
                        point.threshold = thresholds[particle]
                        

    # for i, run in enumerate(root.runs[:10]):
    #     for pick in run.picks:
    #             if pick.points is not None:
    #                 print(pick.points)
    